In [1]:
using LinearAlgebra, Combinatorics

# Problem 1
## Part A
Determine the layout of the masses within the allowable volume. Give a sketch or plot of the layout and a table showing the position the masses, their labels, and positions. Note that you are only deciding on the placement of the masses. You do not have to design the rod arrangement to satisfy any sort of structural requirement (although you may sketch out the connections if you wish). Provide a brief explanation justifying your design.

In [2]:
masses = [8, 1, 2, 8, 4, 1, 8, 7]

side_lengths = [2.9000, 1.8500, 2.1500]

corner_positions = [
         0         0         0
    2.9000         0         0
    2.9000    1.8500         0
         0    1.8500         0
         0         0    2.1500
    2.9000         0    2.1500
    2.9000    1.8500    2.1500
         0    1.8500    2.1500
]

8×3 Matrix{Float64}:
 0.0  0.0   0.0
 2.9  0.0   0.0
 2.9  1.85  0.0
 0.0  1.85  0.0
 0.0  0.0   2.15
 2.9  0.0   2.15
 2.9  1.85  2.15
 0.0  1.85  2.15

In [3]:
# center of yz plane
1.85/2, 2.15/2

(0.925, 1.075)

In [4]:
# function to get coordinates of CoM
function center_of_mass(positions, masses)
    pos_mas_matrix = positions .* masses
    com_pos = [sum(pos_mas_matrix[1:end, i]) / sum(masses) for i ∈ 1:3]
    
    return com_pos
end

# function to get mean deviation from center of yz plane
function yz_mean_deviation(y, z; y_c = 0.925, z_c = 1.075)
    Δy = abs(y - y_c)
    Δz = abs(z - z_c)
    
    mean_deviation = (Δy + Δz) / 2
    
    return mean_deviation    
end

yz_mean_deviation (generic function with 1 method)

In [5]:
# generating permutations of mass combinations
perms = unique(permutations(masses))
coms = [center_of_mass(corner_positions, perm) for perm in perms]

# finding deviations of CoM positions from center of yz plane
deviations = [yz_mean_deviation(com[2], com[3]) for com in coms];

## Part B
Using the external frame, $B$, (origin and axes shown in the figure), calculate the location of the centre of mass.

In [6]:
masses_reordered = [1, 8, 8, 2, 4, 7, 8, 1]
perm_index = 1326

pos_mas_matrix = corner_positions .* masses_reordered

8×3 Matrix{Float64}:
  0.0   0.0    0.0
 23.2   0.0    0.0
 23.2  14.8    0.0
  0.0   3.7    0.0
  0.0   0.0    8.6
 20.3   0.0   15.05
 23.2  14.8   17.2
  0.0   1.85   2.15

In [7]:
com_pos = [sum(pos_mas_matrix[1:end, i]) / sum(masses_reordered) for i ∈ 1:3]

3-element Vector{Float64}:
 2.3051282051282054
 0.9012820512820513
 1.1025641025641024

## Part C
Calculate the inertia matrix $J_B^\prime$. The axes of $B^\prime$ are aligned with $B$ but the origin is located at the centre of mass.

In [8]:
pos_Bp = corner_positions .- transpose(com_pos)

8×3 Matrix{Float64}:
 -2.30513   -0.901282  -1.10256
  0.594872  -0.901282  -1.10256
  0.594872   0.948718  -1.10256
 -2.30513    0.948718  -1.10256
 -2.30513   -0.901282   1.04744
  0.594872  -0.901282   1.04744
  0.594872   0.948718   1.04744
 -2.30513    0.948718   1.04744

In [9]:
function cross_mat(vector)
    @assert length(vector) == 3 "Vectors of length 3 required"
    return [
        0 -vector[3] vector[2]
        vector[3] 0 -vector[1]
        -vector[2] vector[1] 0
    ]
end

cross_mat (generic function with 1 method)

In [10]:
Jb = -sum([
    masses_reordered[i] .* cross_mat(pos_Bp[i, 1:end]) * cross_mat(pos_Bp[i, 1:end]) for i ∈ 1:size(pos_Bp, 1)
])

3×3 Matrix{Float64}:
 78.3872   -4.81474   5.59551
 -4.81474  98.5187    2.95763
  5.59551   2.95763  86.8264

## Part D
Find the three principal moments of inertia for this body, and the directions of the principal axes. On a diagram of the body, sketch the directions of the major and minor axes (and label them).

In [11]:
evals = eigvals(Jb)

3-element Vector{Float64}:
 74.2901726807238
 89.59269716317893
 99.849437848405

In [28]:
evecs = eigvecs(Jb)

3×3 Matrix{Float64}:
 0.896721  -0.00248789   0.44259
 0.42935   -0.237932    -0.871233
 0.107474   0.971279    -0.21229

Checking eigenvalue/eigenvector problem condition (all following vectors should equal to ~0):

In [29]:
((evals[1] * I(3) - Jb) * evecs)[1:end, 1]

3-element Vector{Float64}:
 -2.1649348980190553e-14
 -3.885780586188048e-16
  1.687538997430238e-14

In [30]:
((evals[2] * I(3) - Jb) * evecs)[1:end, 2]

3-element Vector{Float64}:
  1.7763568394002505e-14
  5.773159728050814e-15
 -3.064215547965432e-14

In [31]:
((evals[3] * I(3) - Jb) * evecs)[1:end, 3]

3-element Vector{Float64}:
  1.709743457922741e-14
 -6.106226635438361e-14
 -6.217248937900877e-15

In [32]:
principal_moi = [
    evals[1] 0 0
    0 evals[2] 0 
    0 0 evals[3]
]

3×3 Matrix{Float64}:
 62.6477    0.0      0.0
  0.0     114.155    0.0
  0.0       0.0    142.996

In [33]:
principal_moi_dirs = principal_moi ./ norm(principal_moi)

3×3 Matrix{Float64}:
 0.323926  0.0       0.0
 0.0       0.590249  0.0
 0.0       0.0       0.739377

In [34]:
C_Bp_B = transpose(evecs)

3×3 transpose(::Matrix{Float64}) with eltype Float64:
  0.896721     0.42935    0.107474
 -0.00248789  -0.237932   0.971279
  0.44259     -0.871233  -0.21229

In [36]:
# checking stability condition
# k = ((J1 - J3) * (J2 - J3)) / (J1 * J2)

J1, J2, J3 = evals

k1 = (J2 - J3) / J1
k3 = (J2 - J1) / J3

k1, k3

(-0.46037732325651715, 0.36019928891241687)

## Part E
Show how this body should be oriented in the orbital frame for stability. Assuming that the spacecraft lies in a 105 min. circular orbit, find the three natural frequencies of oscillation.

# Problem 2
## Part A
Calculate the final spin rate about the $z$-axis

In [3]:
r = 1

ω3 = 2

θ = deg2rad(15)

J = [
    80
    80
    180
]

3-element Vector{Int64}:
  80
  80
 180

In [5]:
ω12 = (J[3] * ω3 * tan(θ)) / J[1]
H = sqrt((J[3] * ω3)^2 + (J[1] * ω12)^2)

ωf = H / J[3]

2.070552360820166

## Part B
Calculate the loss in rotational kinetic energy

In [16]:
ω1 = sqrt(ω12^2 / 2)
ω2 = ω1

0.8526091094167769

In [27]:
ω_i = [ω1, ω2, ω3]

T_i = 0.5 * ω_i' * J * ω_i

3-element Vector{Float64}:
 211.6250231718574
 211.6250231718574
 496.4174575066843

In [28]:
T_i_mag = norm(T_i)

579.6555813432483

In [29]:
ω_f = [0, 0, ωf]
T_f = 0.5 * ω_f' * J * ω_f

3-element Vector{Float64}:
   0.0
   0.0
 385.8468371008166

In [31]:
T_f_mag = norm(T_f)

385.8468371008166

In [32]:
T_i_mag - T_f_mag

193.80874424243171

## Part C
A tangential release yo-yo despin device is also included in the satellite. If the two
yo-yo masses are each 4 kg, what cord length is required to completely despin the satellite.

In [33]:
r * sqrt(1 + J[3] / (2 * 4 * r^2))

4.847679857416329